In [2]:
import riskfolio as rp 
import pandas as pd
import numpy as np
from openbb_terminal.sdk import openbb
import datetime
from dateutil.relativedelta import relativedelta
import gspread
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from gspread_dataframe import get_as_dataframe
import yfinance as yf
import bt 
import matplotlib.pyplot as plt
%matplotlib inline



In [3]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive.file','https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/adamjohnson/Documents/GoogleCloudService/gspread-api-394113-fcd586f615d5.json', scope)

client = gspread.authorize(credentials)

#sheet =client.open('Risk Parity Database')

sheet =client.open_by_key('1_XjLk6Vrz7ht5twTK_jARrNf8pBkgVztr8ft5o8ADow')

sheet_instance = sheet.get_worksheet(2)

data = sheet_instance.get_all_values()

portfolio = pd.DataFrame(data)

headers = portfolio.iloc[0]
portfolio = pd.DataFrame(portfolio.values[1:], columns=headers)

portfolio = portfolio.rename(columns= {'':'Ticker'})

portfolio['Shares'] = portfolio['Shares'].astype(float)

total_shares = portfolio['Shares'].sum()

portfolio['% Allocation'] = (portfolio['Shares'] / total_shares)

portfolio



,Ticker,Shares,% Allocation
0,CEG,2108.0,0.108576
1,AAPL,1727.0,0.088952
2,ULTA,1685.0,0.086789
3,MSFT,1657.0,0.085346
4,RTX,1623.0,0.083595
5,JPM,1321.0,0.068040
6,CVX,1276.0,0.065722
7,PANW,1225.0,0.063096
8,GEHC,1141.0,0.058769
9,AMZN,1080.0,0.055627


In [5]:
end = pd.Timestamp(datetime.date.today())
start = end - relativedelta(years=1)

symbols = portfolio['Ticker'].tolist()

tickers = openbb.stocks.ca.hist(symbols,start, end)

returns = tickers.pct_change()[1:]
returns.dropna(how="any", axis=1, inplace=True)

NaN values found in: GEHC.  Backfilling data

In [7]:
risk_measures = ['MV', 'SLPM', 'CVaR','MAD','FLPM','EVaR','UCI','CDaR','MSV']

weights = pd.DataFrame([])

# Create an instance of the Portfolio class
P = rp.Portfolio(returns=returns,)

# Define constraints
P.assets_stats(method_mu='hist', method_cov='hist', d=0.94)

P.lowerret = 0.0010

for rm in risk_measures:

    w_rp = P.rp_optimization(model='Classic', rm = rm, b=None)
    
    weights = pd.concat([weights, w_rp], axis=1)

weights.columns = risk_measures

weights_deindex = weights.reset_index()

weights_deindex.rename(columns={'index':'Ticker'}, inplace=True )

weights_deindex

,Ticker,MV,SLPM,CVaR,MAD,FLPM,EVaR,UCI,CDaR,MSV
0,CEG,0.081754,0.083421,0.072294,0.086497,0.091340,0.094530,0.180201,0.174497,0.082573
1,AAPL,0.066723,0.068267,0.071159,0.068397,0.068998,0.064604,0.044990,0.035211,0.068343
2,ULTA,0.079715,0.088223,0.104389,0.073745,0.078285,0.100209,0.057370,0.061275,0.085575
3,MSFT,0.072837,0.076009,0.081522,0.074352,0.074941,0.072936,0.054767,0.063055,0.075201
4,RTX,0.078074,0.079938,0.071955,0.077442,0.077519,0.076419,0.050368,0.067861,0.079692
5,JPM,0.084966,0.082813,0.065815,0.081586,0.081568,0.073705,0.081476,0.115929,0.081934
6,CVX,0.072925,0.071335,0.067649,0.078955,0.075128,0.080434,0.142533,0.109345,0.072208
7,PANW,0.059132,0.052052,0.044316,0.064706,0.063679,0.041281,0.039331,0.028133,0.053537
8,GEHC,0.071061,0.075857,0.099158,0.063236,0.066435,0.059462,0.059040,0.051871,0.074557
9,AMZN,0.060388,0.055404,0.049799,0.057176,0.054441,0.055599,0.057343,0.053088,0.056233


In [8]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive.file','https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/adamjohnson/Documents/GoogleCloudService/gspread-api-394113-fcd586f615d5.json', scope)

client = gspread.authorize(credentials)

#sheet =client.open('Risk Parity Database')

sheet =client.open_by_key('1_XjLk6Vrz7ht5twTK_jARrNf8pBkgVztr8ft5o8ADow')

sheet_instance = sheet.get_worksheet(1)

#mapping_list =weights_deindex.values.tolist()

#sheet.values_append('RM Weights!A1',{'valueInputOption' : 'RAW'}, {'values':mapping_list})

set_with_dataframe(sheet_instance,weights_deindex)

In [6]:
stock_data = yf.download(symbols, start= start, end= end)['Adj Close']

stock_data.dropna(how="any", axis=1, inplace=True)

port_allocation = portfolio['% Allocation'].tolist()

port_list = portfolio['% Allocation'].tolist()

port_values = portfolio['% Allocation'].values

# Fetch the S&P 500 index data
sp500_data = yf.download('^GSPC', start= start, end= end)['Adj Close']

sp500_data = sp500_data.to_frame(name='S&P500')


[*********************100%***********************]  15 of 15 completed
[*********************100%***********************]  1 of 1 completed


In [10]:
results = {}

# Assuming stock_data is a DataFrame with historical price data for each asset
# stock_data.columns should match the assets in the weights DataFrame

for rm in risk_measures:
    # Extracting the allocation for the given risk measure
    allocation = weights[rm].values

    # Ensuring the weights sum up to 1 (if they don't due to rounding or other reasons)
    allocation = allocation/allocation.sum()

    # Create the portfolio strategy for the given risk measure
    portfolio_weights = dict(zip(stock_data, allocation))
    
    portfolio_strategy = bt.Strategy( f'Portfolio_{rm}',
                                     algos=[
                                         bt.algos.SelectAll(),
                                         bt.algos.WeighSpecified(**portfolio_weights),
                                         bt.algos.Rebalance()
                                     ])

    # Backtest the portfolio strategy
    portfolio_backtest = bt.Backtest(portfolio_strategy, stock_data)
    res = bt.run(portfolio_backtest)

    # Store the backtest result for the risk measure in the results dictionary
    results[rm] = res

 
stats_dict = {}

for rm, res in results.items():
    # Extracting stats for each backtest. This returns a pandas Series
    stats_series = res.stats
    
    # Store the Series in the dictionary with risk measure as the key
    stats_dict[rm] = stats_series

stats_df = pd.concat(stats_dict, axis=1).T

# Transpose again to swap the index with the columns
stats_df = stats_df.T

stats_df.columns = stats_df.columns.get_level_values(0) 

RM_results = stats_df 


   

In [7]:
# Create the portfolio strategy
portfolio_weights = dict(zip(stock_data, port_values))
portfolio_strategy = bt.Strategy('MyPortfolio', 
                                  algos=[
                                      bt.algos.SelectAll(),
                                      bt.algos.WeighSpecified(**portfolio_weights),
                                      bt.algos.Rebalance()
                                  ])

# Backtest the portfolio strategy
portfolio_backtest = bt.Backtest(portfolio_strategy, stock_data)


In [12]:
# Create and backtest the S&P 500 benchmark
benchmark_strategy = bt.Strategy('S&P500', [bt.algos.RunOnce(),
                                            bt.algos.SelectAll(),
                                            bt.algos.WeighEqually(),
                                            bt.algos.Rebalance()])
benchmark_backtest = bt.Backtest(benchmark_strategy, sp500_data)

In [13]:
# Create and backtest the Equal Allocation benchmark
equal_benchmark_strategy = bt.Strategy('EqualAllocation', [bt.algos.RunOnce(),
                                            bt.algos.SelectAll(),
                                            bt.algos.WeighEqually(),
                                            bt.algos.Rebalance()])
equal_benchmark_backtest = bt.Backtest(equal_benchmark_strategy, stock_data)

In [14]:
# Run the backtests and compare the results
res = bt.run(portfolio_backtest, equal_benchmark_backtest, benchmark_backtest)

# Example: Get the equity curve for the first backtest strategy
equity = res[0].prices

# Calculate the daily percentage returns
daily_returns = equity.pct_change() * 100

stats = res.stats

df_stats = pd.DataFrame(stats)

BM_results = df_stats


In [30]:
portfolio_df = pd.concat([RM_results, BM_results], axis = 1)

portfolio_df = portfolio_df.iloc[:-33]

backtested_portfolio = portfolio_df.iloc[3:]

backtested_portfolio = backtested_portfolio.reset_index()

backtested_portfolio.rename(columns={'index':'Results'}, inplace=True )

backtested_portfolio



,Results,MV,SLPM,CVaR,MAD,FLPM,EVaR,UCI,CDaR,MSV,MyPortfolio,EqualAllocation,S&P500
0,total_return,0.281178,0.281667,0.295482,0.272735,0.275881,0.282634,0.256513,0.250461,0.282383,0.282383,0.357377,0.151152
1,cagr,0.281396,0.281885,0.295712,0.272945,0.276094,0.282852,0.25671,0.250653,0.282602,0.282602,0.357661,0.151263
2,max_drawdown,-0.09805,-0.097486,-0.098512,-0.099876,-0.098865,-0.097965,-0.109657,-0.113581,-0.097052,-0.097052,-0.114903,-0.102707
3,calmar,2.869912,2.891541,3.001784,2.732843,2.792641,2.887291,2.341027,2.206817,2.911868,2.911868,3.112724,1.472761
4,mtd,0.067593,0.067427,0.070068,0.067732,0.068089,0.068287,0.071588,0.064897,0.067657,0.067657,0.095149,0.084257
5,three_month,0.053958,0.056482,0.065566,0.050449,0.051812,0.060166,0.053939,0.048472,0.056062,0.056062,0.078143,0.040635
6,six_month,0.120604,0.125009,0.141706,0.116052,0.118672,0.132833,0.09887,0.089365,0.124581,0.124581,0.159955,0.084728
7,ytd,0.336415,0.338496,0.352876,0.326961,0.331639,0.342107,0.313093,0.310982,0.338813,0.338813,0.413182,0.184242
8,one_year,0.281178,0.281667,0.295482,0.272735,0.275881,0.282634,0.256513,0.250461,0.282383,0.282383,0.357377,0.151152
9,three_year,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive.file','https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/adamjohnson/Documents/GoogleCloudService/gspread-api-394113-fcd586f615d5.json', scope)

client = gspread.authorize(credentials)

#sheet =client.open('Risk Parity Database')

sheet =client.open_by_key('1_XjLk6Vrz7ht5twTK_jARrNf8pBkgVztr8ft5o8ADow')

sheet_instance = sheet.get_worksheet(3)

# set dataframe for initial write the use the appending code below for next iterations 
set_with_dataframe(sheet_instance,backtested_portfolio)

In [16]:

merged_df = portfolio.merge(weights_deindex[['Ticker', 'CVaR']], on='Ticker', how='left')

#sum of total portfolio value
sum = merged_df['Shares'].sum()

weekly_invest = 150

merged_df['CVaR $'] = merged_df['CVaR'] * sum

merged_df['$ Diff'] = merged_df['CVaR $'] - merged_df['Shares']

merged_df['Weekly_Contribution'] = merged_df['CVaR'] * weekly_invest

merged_df['date'] = pd.Timestamp.now().date()

merged_df = merged_df.sort_values(by='$ Diff', ascending = True)

merged_df



,Ticker,Shares,% Allocation,CVaR,CVaR $,$ Diff,Weekly_Contribution,date
0,CEG,2108.0,0.108576,0.072294,1403.589355,-704.410645,10.844110,2023-11-21
7,PANW,1225.0,0.063096,0.044316,860.388172,-364.611828,6.647346,2023-11-21
1,AAPL,1727.0,0.088952,0.071159,1381.560701,-345.439299,10.673917,2023-11-21
4,RTX,1623.0,0.083595,0.071955,1397.013803,-225.986197,10.793308,2023-11-21
13,NOW,857.0,0.044141,0.037348,725.102174,-131.897826,5.602129,2023-11-21
9,AMZN,1080.0,0.055627,0.049799,966.849500,-113.150500,7.469865,2023-11-21
3,MSFT,1657.0,0.085346,0.081522,1582.754805,-74.245195,12.228340,2023-11-21
5,JPM,1321.0,0.068040,0.065815,1277.796312,-43.203688,9.872235,2023-11-21
6,CVX,1276.0,0.065722,0.067649,1313.407932,37.407932,10.147370,2023-11-21
10,LOW,1030.0,0.053052,0.057164,1109.842490,79.842490,8.574627,2023-11-21


In [17]:
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/spreadsheets','https://www.googleapis.com/auth/drive.file','https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('/Users/adamjohnson/Documents/GoogleCloudService/gspread-api-394113-fcd586f615d5.json', scope)

client = gspread.authorize(credentials)

#sheet =client.open('Risk Parity Database')

sheet =client.open_by_key('1_XjLk6Vrz7ht5twTK_jARrNf8pBkgVztr8ft5o8ADow')

sheet_instance = sheet.get_worksheet(0)

# set dataframe for initial write the use the appending code below for next iterations 
#set_with_dataframe(sheet_instance,merged_df)


mapping_list =merged_df.values.tolist()

sheet.values_append('Allocation Records!A1',{'valueInputOption' : 'RAW'}, {'values':mapping_list})





TypeError: Object of type date is not JSON serializable

In [18]:
def calculate_risk_measure_allocation(portfolio_df, risk_measure_df, risk_measure_column, weekly_invest=150):
    # Merge the portfolio dataframe with the risk measure dataframe based on the 'Ticker' column
    merged_df = portfolio_df.merge(risk_measure_df[['Ticker', risk_measure_column]], on='Ticker', how='left')
    
    # Calculate the total portfolio value
    total_value = merged_df['Shares'].sum()
    
    # Calculate the risk measure in dollars
    merged_df[risk_measure_column + ' $'] = merged_df[risk_measure_column] * total_value
    
    # Calculate the dollar difference from portfolio value
    merged_df['$ Diff'] = merged_df[risk_measure_column + ' $'] - merged_df['Shares']
    
    # Calculate the weekly contribution
    merged_df['Weekly_Contribution'] = merged_df[risk_measure_column] * weekly_invest
    
    # Sort the dataframe based on the dollar difference in ascending order
    merged_df = merged_df.sort_values(by='$ Diff', ascending=True)
    
    return merged_df

In [19]:

# Create a dictionary to store the results for each risk measure
results = {}

# Iterate through each risk measure column and calculate allocation
for risk_measure_column in risk_measures:
    result_df = calculate_risk_measure_allocation(portfolio, weights_deindex, risk_measure_column)
    results[risk_measure_column] = result_df


In [20]:
mv_df = results['MV']

mv_df

,Ticker,Shares,% Allocation,MV,MV $,$ Diff,Weekly_Contribution
0,CEG,2108.0,0.108576,0.081754,1587.247718,-520.752282,12.263052
1,AAPL,1727.0,0.088952,0.066723,1295.429396,-431.570604,10.008468
3,MSFT,1657.0,0.085346,0.072837,1414.128427,-242.871573,10.925535
2,ULTA,1685.0,0.086789,0.079715,1547.662490,-137.337510,11.957217
4,RTX,1623.0,0.083595,0.078074,1515.808910,-107.191090,11.711117
7,PANW,1225.0,0.063096,0.059132,1148.038324,-76.961676,8.869727
12,NVDA,1004.0,0.051713,0.052090,1011.336408,7.336408,7.813570
9,AMZN,1080.0,0.055627,0.060388,1172.433056,92.433056,9.058200
6,CVX,1276.0,0.065722,0.072925,1415.831514,139.831514,10.938693
10,LOW,1030.0,0.053052,0.060346,1171.625481,141.625481,9.051961
